# RNN tutorial v2
- Use `tf.nn.dynamic_rnn` with MNIST dataset

In [1]:
import numpy as np
import time
from pprint import pprint

import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

## 0. Load MNIST data

In [2]:
mnist = input_data.read_data_sets('MNIST_idx3/', one_hot=False)

Extracting MNIST_idx3/train-images-idx3-ubyte.gz
Extracting MNIST_idx3/train-labels-idx1-ubyte.gz
Extracting MNIST_idx3/t10k-images-idx3-ubyte.gz
Extracting MNIST_idx3/t10k-labels-idx1-ubyte.gz


In [3]:
X_trn, Y_trn = mnist.train.images, mnist.train.labels
X_val, Y_val = mnist.validation.images, mnist.validation.labels
X_test, Y_test = mnist.test.images, mnist.test.labels

In [4]:
num_trn = Y_trn.shape[0]
num_val = Y_val.shape[0]
num_test = Y_test.shape[0]

print("Number of training points: ", num_trn)
print("Number of validation points: ", num_val)
print("Number of test points: ", num_test)

Number of training points:  55000
Number of validation points:  5000
Number of test points:  10000


In [5]:
dim_X = X_trn.shape[1]
pixel_X = int(np.sqrt(dim_X)) # np.sqrt의 출력이 float32이므로, 이를 int 자료형으로 변경

print("Dimension of X: %d (%d x %d)" % (dim_X, pixel_X, pixel_X))

Dimension of X: 784 (28 x 28)


## 1. Parameters

In [6]:
LEARNING_RATE  = 0.001
N_EPOCHS       = 30
BATCH_SIZE     = 128
DISPLAY_STEP   = 50
VAL_EPOCH      = 5

n_input   = pixel_X
n_steps   = pixel_X
n_hidden  = 64
n_classes = 10

## 2. Placeholders

In [7]:
X = tf.placeholder(tf.float32, [None, dim_X])
Y = tf.placeholder(tf.int32, [None,])

In [8]:
print(X.get_shape())

(?, 784)


In [9]:
# Placeholder인 X의 모양이 [?, 784]
# X를 [?, 28, 28]로 변환한 것이 reshaped_X
reshaped_X = tf.reshape(tensor=X, shape=[-1, n_steps, n_input])

print(X.get_shape())
print(reshaped_X.get_shape())

(?, 784)
(?, 28, 28)


## 3. Define the model

In [10]:
# # Define a LSTM cell
# lstm_cell = rnn.BasicLSTMCell(num_units=n_hidden, 
#                               forget_bias=1.0)

# # Get LSTM cell output

# outputs, _ = tf.nn.dynamic_rnn(cell=lstm_cell, 
#                                inputs=reshaped_X, 
#                                dtype=tf.float32)

In [11]:
# print(outputs)

In [12]:
# last_step = list(outputs.get_shape())[1]
# test = outputs[:, last_step-1, :]
# print(test)

In [13]:
# Define RNN model with tf.contrib.rnn.static_rnn
def RNN_dynamic(X, n_steps, n_hidden, n_classes):
    
    # Reshape the placeholder
    reshaped_X = tf.reshape(tensor=X, shape=[-1, n_steps, n_input], name='reshaped')
    
    # Define a LSTM cell
    lstm_cell = rnn.BasicLSTMCell(num_units=n_hidden, 
                                  forget_bias=1.0)
    
    # Get LSTM cell output
    outputs, _ = tf.nn.dynamic_rnn(cell=lstm_cell, 
                                   inputs=reshaped_X, 
                                   dtype=tf.float32)
    last_step = list(outputs.get_shape())[1]
    last_output = outputs[:, last_step - 1, :]
    
    
    # Define weights and biases
    W = tf.get_variable(name='weights', 
                        shape=[n_hidden, n_classes], 
                        initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name='bias', 
                        shape=[n_classes], 
                        initializer=tf.constant_initializer(0.0))
    
    # Calculate logits
    logits = tf.nn.xw_plus_b(x=last_output, weights=W, biases=b, name='logits')
    
    return logits

In [14]:
# logits: output vector of RNN_static
logits = RNN_dynamic(X, n_steps, n_hidden, n_classes)
print(logits)

Tensor("logits:0", shape=(?, 10), dtype=float32)


In [15]:
# Define cost function and training operator with optimizer
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_op = tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

# Evaluate the output of model
correct_prediction = tf.nn.in_top_k(predictions=logits, targets=Y, k=1)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
# Batch_iterator
def make_batch_iterator(class_train, batch_size, allow_small_batch=True):
    num_points = len(class_train)
    start_idx = list(range(0, num_points, batch_size))
    end_idx = list(range(batch_size, num_points + 1, batch_size))
    if allow_small_batch:
        start_idx.append(end_idx[-1] + 1)
        end_idx.append(num_points)
    return zip(start_idx, end_idx)

# 4. Run the model

In [17]:
start_time = time.time()

In [18]:
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # For-loop with respect to epochs
    for epoch in range(N_EPOCHS):
        
        # Make batch_iterator
        batch_iterator = make_batch_iterator(class_train=Y_trn, 
                                             batch_size=BATCH_SIZE,
                                             allow_small_batch=True)
    
        iteration = 0
        
        # For-loop with batch_iterator
        for start, end in batch_iterator:
            
            # Construct the input batch
            batch_xs = X_trn[start:end]
            batch_ys = Y_trn[start:end]
    
            # Run the training operator, accuracy, cpst
            _, cost_batch, acc_batch = sess.run([train_op, cost, accuracy], feed_dict={X: batch_xs, Y: batch_ys})
            
            if (iteration + 1) % DISPLAY_STEP == 0:
                print("[%3d iter / %3d epoch] cost = %.6f, accuracy = %.6f" % (iteration, epoch, cost_batch, acc_batch))
            
            iteration = iteration + 1
        
        if (epoch + 1) % VAL_EPOCH == 0:
            cost_val, acc_val = sess.run([cost, accuracy], feed_dict={X: X_val, Y: Y_val})
            print()
            print("\t[%3d epoch] validation cost = %.6f, validation accuracy = %.6f" % (epoch, cost_val, acc_val))
            print()
    
    # Test
    cost_test, acc_test = sess.run([cost, accuracy], feed_dict={X: X_test, Y: Y_test})
    print("\n\t[Test] test cost = %.6f, test accuracy = %.6f" % (cost_test, acc_test))

[ 49 iter /   0 epoch] cost = 2.292659, accuracy = 0.171875
[ 99 iter /   0 epoch] cost = 2.231856, accuracy = 0.289062
[149 iter /   0 epoch] cost = 1.489734, accuracy = 0.507812
[199 iter /   0 epoch] cost = 1.285939, accuracy = 0.562500
[249 iter /   0 epoch] cost = 0.770704, accuracy = 0.773438
[299 iter /   0 epoch] cost = 0.619487, accuracy = 0.804688
[349 iter /   0 epoch] cost = 0.677381, accuracy = 0.773438
[399 iter /   0 epoch] cost = 0.505758, accuracy = 0.820312
[ 49 iter /   1 epoch] cost = 0.303259, accuracy = 0.914062
[ 99 iter /   1 epoch] cost = 0.517923, accuracy = 0.812500
[149 iter /   1 epoch] cost = 0.276392, accuracy = 0.914062
[199 iter /   1 epoch] cost = 0.279756, accuracy = 0.929688
[249 iter /   1 epoch] cost = 0.244143, accuracy = 0.906250
[299 iter /   1 epoch] cost = 0.145175, accuracy = 0.953125
[349 iter /   1 epoch] cost = 0.231901, accuracy = 0.914062
[399 iter /   1 epoch] cost = 0.176347, accuracy = 0.945312
[ 49 iter /   2 epoch] cost = 0.062852, 

[299 iter /  16 epoch] cost = 0.003088, accuracy = 1.000000
[349 iter /  16 epoch] cost = 0.009321, accuracy = 1.000000
[399 iter /  16 epoch] cost = 0.004111, accuracy = 1.000000
[ 49 iter /  17 epoch] cost = 0.005597, accuracy = 1.000000
[ 99 iter /  17 epoch] cost = 0.026310, accuracy = 0.992188
[149 iter /  17 epoch] cost = 0.028496, accuracy = 0.992188
[199 iter /  17 epoch] cost = 0.016405, accuracy = 0.992188
[249 iter /  17 epoch] cost = 0.022162, accuracy = 0.984375
[299 iter /  17 epoch] cost = 0.002340, accuracy = 1.000000
[349 iter /  17 epoch] cost = 0.007141, accuracy = 1.000000
[399 iter /  17 epoch] cost = 0.005517, accuracy = 1.000000
[ 49 iter /  18 epoch] cost = 0.010134, accuracy = 0.992188
[ 99 iter /  18 epoch] cost = 0.026420, accuracy = 0.992188
[149 iter /  18 epoch] cost = 0.043251, accuracy = 0.992188
[199 iter /  18 epoch] cost = 0.005914, accuracy = 1.000000
[249 iter /  18 epoch] cost = 0.010418, accuracy = 0.992188
[299 iter /  18 epoch] cost = 0.006936, 

In [19]:
print(time.time() - start_time)

398.3017108440399
